# Script that sets up training repository

Change the settings in the first cell and run all to create the repository and send the invitations.

In [ ]:
import os, sys, json,importlib, base64, time

#This notebook requires you to install https://github.com/PyGithub/PyGithub first
from github import Github, InputGitAuthor

#Add parent folder to library paths
module_path = os.path.abspath(os.path.join('../../Common-Resources'))
if module_path not in sys.path:
    sys.path.append(module_path)
import github_tools as ghTools

In [ ]:
ghTools.getTokenAttribute(attribute="toksen")

In [ ]:
######################################
#Get access token and log in main user

accessToken = ghTools.getTokenAttribute(attribute="token")

#Test if an access token exist, if it does, try to log in user, otherwise throw error
if accessToken:
    ghSession1 = Github(accessToken)
    #get user
    user1 = ghSession1.get_user()
else:
    raise Exception('You must pass a token in the attribute token for user 1')

######################################
#Get access token and log in user 2, 
# user 1 will be used as user 2 as well 
# if there is an error

#Get access token for user 2
accessToken2 = ghTools.getTokenAttribute(attribute="token_user2")

#Test if an access token exist, if it does, try to log in user, use user 1
if accessToken2:
    ghSession2 = Github(accessToken2)
    #get user
    user2 = ghSession2.get_user()
else:
    user2 = user1
    
#Print result
print(accessToken, user1.login)
print(accessToken2, user2.login)

In [ ]:
#ghTools.getInputUser()

## Creating the repo

In [ ]:
#The name of the repository to create, use random 4 letters to get unique url
reponame = "country-facts-test"
repoadress = user1.login+"/"+reponame

#making sure the repo is not already there
try:
    del_repo = ghSession1.get_repo(repoadress)
    print("Delete:", del_repo)
    del_repo.delete()
except:
    print("No repo with that name")
    
#Create the repo
description="Country-Fact repository for DIME Analytics Intro-Git-GitHub-Observer training"
user1_repo = user1.create_repo(reponame, auto_init=True,description=description)
print(user1_repo)

#Invite user 2 and have user 2 to accept the invitation
user2_inviation = user1_repo.add_to_collaborators(user2.login)
user2.accept_invitation(user2_inviation)

user2_repo = ghSession2.get_repo(repoadress)
user2_repo

## Add content to repo

### update README

In [ ]:
#Get the commit sha from where the README was created in the auto_init
contents = user1_repo.get_contents("README.md")

#Read the README file in this repo to get the content for the commit
file_content = open('repo-content-files/README.md', 'rb').read()


#Update the README file
readme_update_result = user1_repo.update_file("/README.md", "Update README.md", file_content,contents.sha, branch="master")

readme_update_sha = readme_update_result["commit"].sha
print(readme_update_result)

### add .gitignore

In [ ]:
#Read the README file in this repo to get the content for the commit
file_content = open("repo-content-files/no.gitignore", 'rb').read()

#Update the README file
create_ignore_result = user1_repo.create_file("/.gitignore", "add a ignore file", file_content, branch="master")

### Start building the country-fact.txt file

In [ ]:
#User 1 author
user1_author = ghTools.getUseOrDefault(attribute="user1_author")

#get country users, or use user1 if they are not added
brazil_user = ghTools.getUseOrDefault(attribute="brazil_user",default_author=user1_author)
spain_user = ghTools.getUseOrDefault(attribute="spain_user",default_author=user1_author)
usa_user = ghTools.getUseOrDefault(attribute="usa_user",default_author=user1_author)
india_user = ghTools.getUseOrDefault(attribute="india_user",default_author=user1_author)
nepal_user = ghTools.getUseOrDefault(attribute="nepal_user",default_author=user1_author)
sweden_user = ghTools.getUseOrDefault(attribute="sweden_user",default_author=user1_author)

In [ ]:
#####
# Create the country-fact file

# Country fact file path and name in repo
country_fact_file = "/facts/countries.do"

#Read the README file in this repo to get the content for the commit
file_content = open("repo-content-files/commit1/country-facts.do", 'rb').read()

#Update the README file
create_result = user1_repo.create_file("/facts/countries.do", "Create country facts file", file_content, branch="master")
    
print(create_result)

#####
# Update the country-fact file

#List of all files to commit
country_fact_commits = [
    ["commit2/country-facts.do","Add country headers",user1_author],
    ["commit3/country-facts.do","Add categories",user1_author],
    ["commit4/country-facts.do","Add facts about Sweden",sweden_user],
    ["commit5/country-facts.do","Update country-facts.txt",brazil_user],
    ["commit6/country-facts.do","Spain",spain_user],
    ["commit7/country-facts.do","Update country-facts.txt",usa_user],
    ["commit8/country-facts.do","Updated India facts",india_user],
    ["commit9/country-facts.do","Update country-facts.txt",nepal_user]
]

#Loop over all commit and commit them
for country_fact_commit in country_fact_commits:
    
    #Get the local content to be committed
    file_content = open("repo-content-files/" + country_fact_commit[0], 'rb').read()

    #Submit the commit
    create_result = user1_repo.update_file(country_fact_file,country_fact_commit[1],file_content,
                                              create_result["content"].sha,branch="master",committer=country_fact_commit[2])
    #Print results
    print(country_fact_commit[2],country_fact_commit[1])
    
last_fact_commit_sha = create_result["commit"].sha
last_fact_content_sha = create_result["content"].sha

## Work with the content

### Create an issue

In [ ]:
#################
# Craete the issue

#Create parameters for issue
title ="Capital of India is incorrect"
code_line_reference_URL = "https://github.com/" +repoadress+ "/blob/" +last_fact_commit_sha+ "/" +country_fact_file+ "#L18-L22"
body = code_line_reference_URL + "\n I do not think that the capital of India is Mumbai. Does anyone know what the real capitol of India is?"

#Create the issue
india_issue_user2 = user2_repo.create_issue(title=title,body=body)
print(india_issue_user2)

#################
# First reply to the issue


india_issue_user1 = user1_repo.get_issue(number=india_issue_user2.number)
#Reply
india_issue_user1.create_comment("I know it! It is **New Dehli!** I will do this!")
#Self assignment
india_issue_user1.add_to_assignees(user1.login)

#Sleep to guarantee that assignee comes bofore next comment to issue
time.sleep(3)

### Create branch where issue will be fixed

In [ ]:
#Create a brnach to be used to fix the issue in india
fix_india_capital_branch_name = "fix-india-capital"
user1_repo.create_git_ref("refs/heads/" + fix_india_capital_branch_name, last_fact_commit_sha)

india_issue_user1.create_comment("I have opened a new branch called `" +fix_india_capital_branch_name+ "`. Follow my progress there.")
#print(resp)

### Commit fix to issue

In [ ]:
#Prepare the correct India fix
country_fact_commit = ["commit-fix-dehli/country-facts.do","Correct India capital #" +str(india_issue_user1.number) ,InputGitAuthor("kbjarkefur","kbrkbr000@gmail.com")]

#Get the local content to be committed
file_content = open("repo-content-files/" + country_fact_commit[0], 'rb').read()

#Submit the commit
create_result = user1_repo.update_file(country_fact_file,country_fact_commit[1],file_content,
                                          last_fact_content_sha,branch=fix_india_capital_branch_name,
                                          committer=country_fact_commit[2])
#Print results
print(country_fact_commit[2])
print(create_result)

#Sleep to guarantee that fix comes bofore next comment to issue
time.sleep(3)

#Comment in the issue that this has been fixed
india_issue_user1.create_comment("I have fixed this in " + str(create_result["commit"].sha) + " and will open a pull request.")

### Start pull request

In [ ]:
#Parameters for Pull request creaetion
title = "Fix India's capital, issue #"+str(india_issue_user1.number)
body = "**Mumbai** was previosuly listed as capital of India. This is now corrected to **New Dehli** \n\n This fixes issue #"+str(india_issue_user1.number)

#Create the pull request
india_PR = user1_repo.create_pull(title=title,body=body,base="master",head=fix_india_capital_branch_name)

#Add a reveiwer
#india_PR.create_review_request([user2.login])
#this does not work in pygithub 1.39 which is the most recent version that works on a Bank device

#Sleep to guarantee that PR is opened before next comment to issue
time.sleep(3)

In [ ]:
#Say thank you from original poster!
india_issue_user2.create_comment("Thank you @" +user1.login+ ", I will review the pull request and merge it!")